### Movenet test 

In [136]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed 
import numpy as np
import cv2
import os
import keras
from sklearn.model_selection import train_test_split


# Matplotlib 
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Animation libraries 
import imageio
from IPython.display import HTML, display

import os
import numpy as np
from PIL import Image

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout

### Load movenet model from TF Hub

In [137]:
model_name = "movenet_lightning"

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

### Load Images from Folders 

In [138]:
def load_all_images():
    # Root directory containing all numbered folders
    root_dir = os.path.join(os.getcwd(), 'TestImages', 'ResizedImages')
    
    # Number of folders (0-15)
    num_folders = 16
    
    # Create lists to store all images and their labels
    all_images = []
    all_labels = []
    
    # Loop through each folder
    for folder_num in range(num_folders):
        folder_path = os.path.join(root_dir, str(folder_num))
        
        # Check if folder exists
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist.")
            continue
            
        # Get all image files from the folder
        image_files = [f for f in os.listdir(folder_path)
                    if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
        
        print(f"Loading {len(image_files)} images from class {folder_num}")
        
        # Load each image
        for img_file in image_files:
            img_path = os.path.join(folder_path, img_file)
            try:
                # Open the image
                img = Image.open(img_path)
                
                # Convert to numpy array
                img_array = np.array(img)
                
                # Add to the lists
                all_images.append(img_array)
                all_labels.append(folder_num)  # Use folder number as label
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    
    # Convert lists to numpy arrays
    X = np.array(all_images)
    y = np.array(all_labels)
    
    print(f"Total images loaded: {len(all_images)}")
    print(f"Images shape: {X.shape}")
    print(f"Labels shape: {y.shape}")
    
    return X, y  # Return both images and labels

# Load all images and labels
images, labels = load_all_images()

Loading 24 images from class 0
Loading 36 images from class 1
Loading 12 images from class 2
Loading 36 images from class 3
Loading 12 images from class 4
Loading 24 images from class 5
Loading 24 images from class 6
Loading 24 images from class 7
Loading 36 images from class 8
Loading 12 images from class 9
Loading 36 images from class 10
Loading 12 images from class 11
Loading 24 images from class 12
Loading 24 images from class 13
Loading 36 images from class 14
Loading 12 images from class 15
Total images loaded: 384
Images shape: (384, 1000, 660, 3)
Labels shape: (384,)


### Predict Image 

In [164]:
# Extract the first image and label
test_image = images[0]
test_label = labels[0]

# Remove the first image
train_images = images[1:]
train_labels = labels[1:]

temp = tf.expand_dims(test_image, axis=0)
temp = tf.image.resize_with_pad(temp, input_size, input_size)
predict_keypoints = movenet(temp)

predict_keypoints = np.array(predict_keypoints)
predict_keypoints = predict_keypoints[0, 0]  # Now has shape (17, 3)


predict_keypoints.shape


(17, 3)

### Apply Movenet to images to get the key points 

In [141]:
# Resize and pad the image to keep the aspect ratio and fit the expected size.

def keypoints(images):
  keypoints_with_scores = []

  for image in images:
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    keypoints_with_scores.append(movenet(input_image))


  return keypoints_with_scores


movenet_keypoints = keypoints(images=images)

movenet_keypoints



[array([[[[0.13521795, 0.4916587 , 0.767021  ],
          [0.10811444, 0.5208181 , 0.49992085],
          [0.1038027 , 0.4645365 , 0.55294085],
          [0.12931561, 0.55480844, 0.74809253],
          [0.12089732, 0.42535716, 0.57911795],
          [0.31035772, 0.62138903, 0.8153109 ],
          [0.29797566, 0.33143154, 0.7452317 ],
          [0.5697316 , 0.61648124, 0.65618736],
          [0.54801685, 0.2814397 , 0.869993  ],
          [0.46754977, 0.6630864 , 0.769563  ],
          [0.46669924, 0.3318255 , 0.7324631 ],
          [0.75729936, 0.56236225, 0.5930226 ],
          [0.75808895, 0.37412697, 0.59967136],
          [1.0096247 , 0.5951579 , 0.12281527],
          [0.9955334 , 0.3411083 , 0.2842766 ],
          [0.9866719 , 0.36596113, 0.04031428],
          [0.99750894, 0.33350933, 0.07160485]]]], dtype=float32),
 array([[[[0.1425851 , 0.46509394, 0.69782394],
          [0.1174522 , 0.4941325 , 0.82675546],
          [0.11353093, 0.4383859 , 0.69209206],
          [0.13424051

### Convert to numPy array and remove unnessasary dimensions

In [143]:
keypoints_array = np.array(movenet_keypoints)
keypoints_array = keypoints_array.squeeze(axis=(1,2))

keypoints_array.shape


(384, 17, 3)

### One hot encoded feature Data 

In [146]:
label_data_hot = keras.utils.to_categorical(labels, num_classes=16)

label_data_hot

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

### Neural Networks - Multilayered 

In [147]:
X_train, X_test, y_train, y_test = train_test_split(keypoints_array, label_data_hot, test_size=0.2, random_state=42)

model = Sequential()
model.add(Flatten(input_shape = (17,3)))
model.add(Dense(units=64, activation='relu'))

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2)) 
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.3)) 
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=2048, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=label_data_hot.shape[1], activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=200, batch_size=35)

Epoch 1/200


c:\Users\Georg\anaconda3\envs\steam_analysis\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.0678 - loss: 2.7462
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1639 - loss: 2.6618
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1978 - loss: 2.4848
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2344 - loss: 2.2516
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3035 - loss: 2.0040
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3554 - loss: 1.7189
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3573 - loss: 1.6488
Epoch 8/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4396 - loss: 1.4720
Epoch 9/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4464 - loss: 1.4890
Epoch 10/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4420 - loss: 1.4043
Epoch 11/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4597 - loss: 1.3394
Epoch 12/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4542 - loss: 1.3267
E

In [148]:
loss, accuracy = model.evaluate(X_test, y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7296 - loss: 0.9628 


In [170]:
predict_keypoints_batch = np.expand_dims(predict_keypoints, axis=0)
model.predict(predict_keypoints_batch)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


array([[9.9889034e-01, 1.2007522e-09, 3.3135317e-12, 2.8926070e-08,
        1.7535574e-05, 1.4496265e-10, 7.4191587e-07, 1.0443351e-03,
        9.0800178e-10, 3.2668912e-09, 1.0017812e-07, 2.2882528e-05,
        2.3737470e-05, 2.4343598e-07, 1.8422798e-11, 1.7606437e-11]],
      dtype=float32)